In [51]:
# Sentence Transformer Training
from sentence_transformers import SentenceTransformer, models, util, losses, datasets
from sentence_transformers.readers import InputExample
from transformers import BertTokenizer, BertForMaskedLM, AutoModel, AutoTokenizer
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
# import datasets

In [6]:
import azureml
from azureml.core import Workspace, Datastore, Dataset
from azureml.data.data_reference import DataReference
import pandas as pd
import tempfile
import torch
import sentence_transformers
from sentence_transformers import SentenceTransformer,CrossEncoder, util

import numpy as np
import pickle
import time
import joblib

In [7]:
from azure.ai.ml.constants import AssetTypes 
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)

from azure.identity import DefaultAzureCredential, ManagedIdentityCredential

In [87]:
bert = models.Transformer('sentence-transformers/msmarco-distilbert-base-v4')

pooling_model = models.Pooling(
    bert.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True
)

mymodel = SentenceTransformer(modules=[bert, pooling_model])
mymodel

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [92]:
for name, param in mymodel.named_parameters():
    print(name)

0.auto_model.embeddings.word_embeddings.weight
0.auto_model.embeddings.position_embeddings.weight
0.auto_model.embeddings.LayerNorm.weight
0.auto_model.embeddings.LayerNorm.bias
0.auto_model.transformer.layer.0.attention.q_lin.weight
0.auto_model.transformer.layer.0.attention.q_lin.bias
0.auto_model.transformer.layer.0.attention.k_lin.weight
0.auto_model.transformer.layer.0.attention.k_lin.bias
0.auto_model.transformer.layer.0.attention.v_lin.weight
0.auto_model.transformer.layer.0.attention.v_lin.bias
0.auto_model.transformer.layer.0.attention.out_lin.weight
0.auto_model.transformer.layer.0.attention.out_lin.bias
0.auto_model.transformer.layer.0.sa_layer_norm.weight
0.auto_model.transformer.layer.0.sa_layer_norm.bias
0.auto_model.transformer.layer.0.ffn.lin1.weight
0.auto_model.transformer.layer.0.ffn.lin1.bias
0.auto_model.transformer.layer.0.ffn.lin2.weight
0.auto_model.transformer.layer.0.ffn.lin2.bias
0.auto_model.transformer.layer.0.output_layer_norm.weight
0.auto_model.transform

In [93]:
for name, param in mymodel.named_parameters():
    print(name)

0.auto_model.embeddings.word_embeddings.weight
0.auto_model.embeddings.position_embeddings.weight
0.auto_model.embeddings.LayerNorm.weight
0.auto_model.embeddings.LayerNorm.bias
0.auto_model.transformer.layer.0.attention.q_lin.weight
0.auto_model.transformer.layer.0.attention.q_lin.bias
0.auto_model.transformer.layer.0.attention.k_lin.weight
0.auto_model.transformer.layer.0.attention.k_lin.bias
0.auto_model.transformer.layer.0.attention.v_lin.weight
0.auto_model.transformer.layer.0.attention.v_lin.bias
0.auto_model.transformer.layer.0.attention.out_lin.weight
0.auto_model.transformer.layer.0.attention.out_lin.bias
0.auto_model.transformer.layer.0.sa_layer_norm.weight
0.auto_model.transformer.layer.0.sa_layer_norm.bias
0.auto_model.transformer.layer.0.ffn.lin1.weight
0.auto_model.transformer.layer.0.ffn.lin1.bias
0.auto_model.transformer.layer.0.ffn.lin2.weight
0.auto_model.transformer.layer.0.ffn.lin2.bias
0.auto_model.transformer.layer.0.output_layer_norm.weight
0.auto_model.transform

In [117]:
mymodel[0], mymodel[1]

(Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: DistilBertModel ,
 Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False}))

In [124]:
# Freeze all the parameters
for name, param in mymodel[0].named_parameters():
    param.requires_grad = False

# Unfreeze the last few layers
for name, param in mymodel[0].named_parameters():
    if (name.__contains__('layer.5')):
        print(name)
        param.requires_grad = True
    

auto_model.transformer.layer.5.attention.q_lin.weight
auto_model.transformer.layer.5.attention.q_lin.bias
auto_model.transformer.layer.5.attention.k_lin.weight
auto_model.transformer.layer.5.attention.k_lin.bias
auto_model.transformer.layer.5.attention.v_lin.weight
auto_model.transformer.layer.5.attention.v_lin.bias
auto_model.transformer.layer.5.attention.out_lin.weight
auto_model.transformer.layer.5.attention.out_lin.bias
auto_model.transformer.layer.5.sa_layer_norm.weight
auto_model.transformer.layer.5.sa_layer_norm.bias
auto_model.transformer.layer.5.ffn.lin1.weight
auto_model.transformer.layer.5.ffn.lin1.bias
auto_model.transformer.layer.5.ffn.lin2.weight
auto_model.transformer.layer.5.ffn.lin2.bias
auto_model.transformer.layer.5.output_layer_norm.weight
auto_model.transformer.layer.5.output_layer_norm.bias


In [8]:
subscription_id_ = "10261ac3-459c-4a4d-bbb3-b4099e8d76b0"
resource_group_ = "mlopsdev"
workspace_ = "cgamlopssecondary"


ml_client = MLClient(
    DefaultAzureCredential(), 
    subscription_id_, 
    resource_group_, 
    workspace_,
    )

Class WorkspaceHubOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [9]:
ws = Workspace.from_config()

In [10]:
ws.datastores['workspaceadlsstore']

{
  "name": "workspaceadlsstore",
  "container_name": "models",
  "account_name": "cgaadlsdev",
  "protocol": "https",
  "endpoint": "core.windows.net"
}

In [14]:
datastore = ws.datastores['workspaceadlsstore']

datastore_paths = [(datastore, 'msx_copilot_content_recommendation/')]
dataset = Dataset.File.from_files(path=datastore_paths)

In [15]:
mounted_path = tempfile.mkdtemp()

# mount dataset onto the mounted_path of a Linux-based compute
mount_context = dataset.mount(mounted_path)

mount_context.start()

import os
print(os.listdir(mounted_path))
print (mounted_path)

['ContentQuestions_FinalDataset.csv', 'conda_yaml', 'input', 'logs', 'models', 'output']
/tmp/tmp9oieszua


In [16]:
df = pd.read_csv(mounted_path + '/ContentQuestions_FinalDataset.csv',
                                       low_memory=False)



In [17]:
df.shape

(20604, 5)

In [38]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows',100)
pd.set_option('display.max_columns',100)

In [40]:
df['questions_cleaned']=df['question'].apply(lambda x: x[8:-2])

In [43]:
df.head(2)

,Unnamed: 0,content,question,label,counter,questions_cleaned
0,0,"006 Email Template: MCEM2_MTB_Comprehensive Threat Protection with XDR and SIEM_EN-NA_Boston_020724. Use this email template to generate interest with your EXISTING CUSTOMERS ONLY. Email blast use is limited to existing customer relationships.. SMC Commercial, SMC Government, Major Public Sector, Strategic Commercial, Strategic Public Sector, SMC Education, Major Commercial. Security.","""1"": ""What specific features of XDR and SIEM are highlighted in the MCEM2_MTB email template for enhancing threat protection?"",",1,1,What specific features of XDR and SIEM are highlighted in the MCEM2_MTB email template for enhancing threat protection?
1,1,"006 Email Template: MCEM2_MTB_Comprehensive Threat Protection with XDR and SIEM_EN-NA_Boston_020724. Use this email template to generate interest with your EXISTING CUSTOMERS ONLY. Email blast use is limited to existing customer relationships.. SMC Commercial, SMC Government, Major Public Sector, Strategic Commercial, Strategic Public Sector, SMC Education, Major Commercial. Security.","""2"": ""How does the MCEM2_MTB email template propose to integrate Comprehensive Threat Protection with existing systems in our organization?"",",1,1,How does the MCEM2_MTB email template propose to integrate Comprehensive Threat Protection with existing systems in our organization?


In [42]:
# df['question'][2789][8:-2]

In [44]:
df['content'][0]

'006 Email Template: MCEM2_MTB_Comprehensive Threat Protection with XDR and SIEM_EN-NA_Boston_020724. Use this email template to generate interest with your EXISTING CUSTOMERS ONLY.  Email blast use is limited to existing customer relationships.. SMC Commercial, SMC Government, Major Public Sector, Strategic Commercial, Strategic Public Sector, SMC Education, Major Commercial. Security.'

In [65]:
samples = []
for i in range(0,df.shape[0]):
    samples.append(InputExample(
        texts=[df['content'][i], df['questions_cleaned'][i]],
        label=df['label'][i]
    ))

In [66]:
len(samples)

20604

In [67]:
batch_size = 8

In [68]:
loader = datasets.NoDuplicatesDataLoader(
    samples, batch_size = batch_size
)

In [125]:
loss = losses.ContrastiveLoss(mymodel)

In [135]:
epochs = 2
warmup = int(.0015*len(loader))
warmup

3

In [136]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(device)
else:
    device = "cpu"
    print(device)

cpu


In [137]:
mymodel.to(device)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [138]:
import os
os.getcwd()

'/home/azureuser/localfiles/MLOps/AML/msx_copilot_content_recomm'

In [139]:
mymodel.fit(
    train_objectives = [(loader, loss)],
    epochs=epochs,
    warmup_steps = warmup,
    output_path='ms_marco_finetuned',
    show_progress_bar=False
)